# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [1]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [2]:
# Lire les données du fichier que vous venez d'enregistrer

books_df = pd.read_csv('resultats.csv')

print(books_df.head())  # Afficher les premières lignes du DataFrame

                                   title   price rating availability
0                   A Light in the Attic  £51.77  Three     In stock
1                     Tipping the Velvet  £53.74    One     In stock
2                             Soumission  £50.10    One     In stock
3                          Sharp Objects  £47.82   Four     In stock
4  Sapiens: A Brief History of Humankind  £54.23   Five     In stock


## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [3]:
# Vérification des types de données

print (books_df.dtypes) 

title           object
price           object
rating          object
availability    object
dtype: object


Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [4]:
# # Conversion de title en chaîne de caractères
# books_df["title"].astype(str)

# # Vérification du type de la colonne title
# print(books_df["title"].dtype)

Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [ ]:
# # Convertir la colonne price en type décimal
# books_df["price"] = books_df["price"].str.replace("£", "").astype(float)

# # Vérification du type de la colonne price
# print(books_df["price"].dtype)

float64


Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [6]:
# # # Valeurs possibles de la colonne availability
# books_df["availability"] = books_df["availability"].str.replace("In stock", "True/False").astype(bool)

# print (books_df["availability"].dtype)

Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [7]:
# Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """
    
    if value == "In stock" :
        return True
    return False

    # Vérification des types de données après conversion
print(books_df.dtypes)


title           object
price           object
rating          object
availability    object
dtype: object


Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [8]:
# # Convertir la colonne availability en booléen (True/False)
# books_df["availability"] = books_df["availability"].apply(convert_availability)

# # Vérification du type de la colonne availability
# print(books_df["availability"].dtype)

Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [9]:
# Dictionnaire associant les notes au format initial et les valeurs numérique
# ratings_map = {
#     "one": 1,
#     "two": 2,
#     "three": 3,
#     "four": 4,
#     "five": 5,
# }
# books_df["rating"] = books_df["rating"].map(ratings_map)
# books_df["rating"] = books_df["rating"].astype(float) # Convertir en float

# Vérification du type de la colonne rating
# print(books_df["rating"].dtype)

In [10]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(books_df: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        books_df (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """
    books_df["title"] = books_df["title"].astype(str)
    books_df["price"] = books_df["price"].str.replace("£", "").astype(float)
    books_df["availability"] = books_df["availability"].apply(convert_availability)
    books_df["availability"] = books_df["availability"].str.replace("In stock", "True/False").astype(bool)
    ratings_map = {
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5
    }
    books_df["rating"] = books_df["rating"].map(ratings_map)
    books_df["rating"] = books_df["rating"].astype(float) # Convertir en float

    return books_df
print(books_df.dtypes)
print(books_df.head())  



title           object
price           object
rating          object
availability    object
dtype: object
                                   title   price rating availability
0                   A Light in the Attic  £51.77  Three     In stock
1                     Tipping the Velvet  £53.74    One     In stock
2                             Soumission  £50.10    One     In stock
3                          Sharp Objects  £47.82   Four     In stock
4  Sapiens: A Brief History of Humankind  £54.23   Five     In stock


---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [11]:
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import requests

# Exemple de data propre
def get_books_from_page(soup: BeautifulSoup) -> list[BeautifulSoup]:
    return soup.find_all("article", class_="product_pod")
def extract_title(book: BeautifulSoup) -> str:
    return book.find("h3").find("a")["title"]
def extract_price(book: BeautifulSoup) -> str:
    return book.find("p", class_="price_color").text
def extract_rating(book: BeautifulSoup) -> str:
    return book.find("p", class_="star-rating")["class"][1]
def extract_availability(book: BeautifulSoup) -> str:
    return book.find("p", class_="instock availability").text.strip()

def extract_book_info(book: BeautifulSoup) -> dict:
    return {
        "title": extract_title(book),
        "price": extract_price(book),
        "rating": extract_rating(book),
        "availability": extract_availability(book)
    }

base_url = "http://books.toscrape.com/catalogue/page-{}.html"

def scrape_books(pages: int) -> list[dict]:
    all_books = []
# Scraping sur plusieurs pages
    for page in range(1, pages + 1):
        url = base_url.format(page)
        print(f"Scraping page {page}: {url}")
        response = requests.get(url)

        if response.status_code == 404:
            print(f" Page {page} not found (404). Stopping scrape.")
            break  # Arrêter la boucle si la page n’existe pas

        soup = BeautifulSoup(response.content, "html.parser")
        books_tags = get_books_from_page(soup)

        if not books_tags:
            print(f"Aucune donnée sur la page {page}. Fin du scraping.")
            break

        data_books = [extract_book_info(book) for book in books_tags]
        all_books.extend(data_books)

    return all_books
books_data = scrape_books(50)  

books_df = pd.DataFrame(books_data)

# Nettoyage préventif
books_df.columns = [str(col).strip().replace(" ", "_").replace("-", "_") for col in books_df.columns]
books_df = books_df.loc[:, books_df.columns.notnull()]

# Connexion et insertion
conn = sqlite3.connect("books.db")
books_df.to_sql("book_bdd_1", conn, if_exists="replace", index=False)
conn.commit()



Scraping page 1: http://books.toscrape.com/catalogue/page-1.html
Scraping page 2: http://books.toscrape.com/catalogue/page-2.html
Scraping page 3: http://books.toscrape.com/catalogue/page-3.html
Scraping page 4: http://books.toscrape.com/catalogue/page-4.html
Scraping page 5: http://books.toscrape.com/catalogue/page-5.html
Scraping page 6: http://books.toscrape.com/catalogue/page-6.html
Scraping page 7: http://books.toscrape.com/catalogue/page-7.html
Scraping page 8: http://books.toscrape.com/catalogue/page-8.html
Scraping page 9: http://books.toscrape.com/catalogue/page-9.html
Scraping page 10: http://books.toscrape.com/catalogue/page-10.html
Scraping page 11: http://books.toscrape.com/catalogue/page-11.html
Scraping page 12: http://books.toscrape.com/catalogue/page-12.html
Scraping page 13: http://books.toscrape.com/catalogue/page-13.html
Scraping page 14: http://books.toscrape.com/catalogue/page-14.html
Scraping page 15: http://books.toscrape.com/catalogue/page-15.html
Scraping page

In [12]:
import sqlite3
import pandas as pd

# Connexion à la BDD
conn = sqlite3.connect("book_store.db")
# Insertion dans la table
books_df.to_sql("book_bdd_1", conn, if_exists="replace", index=False)

1000

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [15]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect("books.db")
cursor = conn.cursor()

# Requête SQL pour compter le nombre de livres
cursor.execute("SELECT COUNT(*) FROM book_bdd_1")

# Récupération du résultat
nb_livres = cursor.fetchone()[0]

# Affichage
print(f" Nombre de livres dans la base de données : {nb_livres}")



 Nombre de livres dans la base de données : 1000


In [14]:
print(books_df.shape)


(1000, 4)
